# Reinforcement Learning Final Project: Exploration Strategies for Robotic Manipulation

### Import helper libraries

In [13]:
### Important trick to keep JupyterLab from crashing
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import gym
import sys
import pickle
import traceback
from arguments import get_args
import random
import torch
import tqdm
from datetime import datetime
from mpi4py import MPI
from mpi_utils.mpi_utils import sync_networks, sync_grads
from rl_modules.replay_buffer import replay_buffer
from rl_modules.models import actor, critic, noisy_actor, noisy_critic
from mpi_utils.normalizer import normalizer
from her_modules.her import her_sampler
import copy
import math
from typing import Dict, List, Tuple, Callable
from collections import namedtuple
from copy import deepcopy
import ipywidgets as widgets
import matplotlib.pyplot as plt
import more_itertools as mitt
import pygame
import glfw
from math import floor
from tiling import IHT
from pathlib import Path
from schedule import ExponentialSchedule, OUSchedule

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [36, 4]
fullPath = str(Path('.').absolute())

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['IN_MPI'] = '1'

### Create environments and place to store model weights
envs = {
    'push': {
        'model': gym.make('FetchPush-v1'),
        'weights': None
    },
    'reach': {
        'model': gym.make('FetchReach-v1'),
        'weights': None
    },
    'slide': {
        'model': gym.make('FetchSlide-v1'),
        'weights': None
    },
    'pick': {
        'model': gym.make('FetchPickAndPlace-v1'),
        'weights': None
    }
}

### Graphically Render Policy

Using Mujoco backend

In [14]:
def render(env, policy=None):
    """Graphically render an episode using the given policy

    :param env:  Gym environment
    :param policy:  function which maps state to action.  If None, the random
                    policy is used.
    """
    glfw.init()
    if policy is None:

        def policy(state):
            return env.action_space.sample()

    state = env.reset()
    env.render()
    i = 0
    while i < 3000:
        action = policy(state)
        state, _, done, _ = env.step(action)
        env.render()
        if done:
            break
        i += 1
            
    env.close()
    glfw.terminate()

### Helper Functions

In [15]:
def preproc_og(o, g):
    o = np.clip(o, -200, 200)
    g = np.clip(g, -200, 200)
    return o, g

# pre_process the inputs
def preproc_inputs(obs, g):
    obs_norm = self.o_norm.normalize(obs)
    g_norm = self.g_norm.normalize(g)
    # concatenate the stuffs
    inputs = np.concatenate([obs_norm, g_norm])
    inputs = torch.tensor(inputs, dtype=torch.float32).unsqueeze(0)
    return inputs

### Define Structure for Deep Deterministic Policy Gradient (DDPG) Agent

In [16]:
"""
ddpg with HER (MPI-version)

"""
class ddpg_agent:
    def __init__(self, args, env, env_params):
        self.args = args
        self.env = env
        self.env_params = env_params
        # create the network
        self.actor_network = noisy_actor(env_params) if bool(self.args.noisy) else actor(env_params)
        self.critic_network = noisy_critic(env_params) if bool(self.args.noisy) else critic(env_params) 
        # sync the networks across the cpus
        sync_networks(self.actor_network)
        sync_networks(self.critic_network)
        # build up the target network
        self.actor_target_network = actor(env_params)
        self.critic_target_network = critic(env_params)
        # load the weights into the target networks
        self.actor_target_network.load_state_dict(self.actor_network.state_dict())
        self.critic_target_network.load_state_dict(self.critic_network.state_dict())
        # if use gpu
        if self.args.cuda:
            self.actor_network.cuda()
            self.critic_network.cuda()
            self.actor_target_network.cuda()
            self.critic_target_network.cuda()
        # create the optimizer
        self.actor_optim = torch.optim.Adam(self.actor_network.parameters(), lr=self.args.lr_actor)
        self.critic_optim = torch.optim.Adam(self.critic_network.parameters(), lr=self.args.lr_critic)
        # her sampler
        self.her_module = her_sampler(self.args.replay_strategy, self.args.replay_k, self.env.compute_reward)
        # create the replay buffer
        self.buffer = replay_buffer(self.env_params, self.args.buffer_size, self.her_module.sample_her_transitions)
        # create the normalizer
        self.o_norm = normalizer(size=env_params['obs'], default_clip_range=self.args.clip_range)
        self.g_norm = normalizer(size=env_params['goal'], default_clip_range=self.args.clip_range)
        ########### Setup exploration parameters ########
        # Create exploration parameters
        self.epsilon = self.args.epsilon
        # Create e-greedy schedule
        self.e_schedule = ExponentialSchedule(1.0,0.01,self.args.n_cycles)
        self.ou_schedule = OUSchedule(self.args.n_cycles,4)
        # Create store for count based methods
        self.num_tiles = 10
        self.max_size = 1000
        self.iht = IHT(self.max_size)
        self.weights_fq = np.zeros(self.max_size)
        self.weights_ucb = np.zeros(self.max_size)
        # Create strategy store
        self.strategies = {
        'no_explore': self.no_explore,
        'standard': self.standard_explore,
        'e_greedy': self.e_greedy,
        'e_greedy_decay': self.e_greedy_decay,
        'ucb': self.ucb,
        'ou': self.ornstein_uhlenbeck,
        'count': self.frequency_explore
        }
        # Select the exploration strategy
        self.explore_strategy = self.strategies[self.args.strategy]

    def learn(self):
        """
        train the network

        """
        # start to collect samples
        pbar = tqdm.notebook.trange(self.args.n_epochs)
        models = []
        accuracies = []
        o_norms = []
        g_norms = []
        self.iht = IHT(self.max_size)
        self.weights_fq = np.zeros(self.max_size)
        self.weights_ucb = np.zeros(self.max_size)
        for epoch in pbar:
            pbar.set_description(f'Executing epoch: {epoch}/{self.args.n_epochs}')
            # Reset process noise
            self.ou_schedule.reset()
            # Reset count based method stores
            for cycle_num in range(self.args.n_cycles):
                mb_obs, mb_ag, mb_g, mb_actions = [], [], [], []
                for _ in range(self.args.num_rollouts_per_mpi):
                    # reset the rollouts
                    ep_obs, ep_ag, ep_g, ep_actions = [], [], [], []
                    # reset the environment
                    observation = self.env.reset()
                    obs = observation['observation']
                    ag = observation['achieved_goal']
                    g = observation['desired_goal']
                    # start to collect samples
                    for t in range(self.env_params['max_timesteps']):
                        with torch.no_grad():
                            input_tensor = self._preproc_inputs(obs, g)
                            pi = self.actor_network(input_tensor)
                            action = self._select_actions(pi, input_tensor, cycle_num)
                        # feed the actions into the environment
                        observation_new, _, _, info = self.env.step(action)
                        obs_new = observation_new['observation']
                        ag_new = observation_new['achieved_goal']
                        # append rollouts
                        ep_obs.append(obs.copy())
                        ep_ag.append(ag.copy())
                        ep_g.append(g.copy())
                        ep_actions.append(action.copy())
                        # re-assign the observation
                        obs = obs_new
                        ag = ag_new
                    ep_obs.append(obs.copy())
                    ep_ag.append(ag.copy())
                    mb_obs.append(ep_obs)
                    mb_ag.append(ep_ag)
                    mb_g.append(ep_g)
                    mb_actions.append(ep_actions)
                # convert them into arrays
                mb_obs = np.array(mb_obs)
                mb_ag = np.array(mb_ag)
                mb_g = np.array(mb_g)
                mb_actions = np.array(mb_actions)
                # store the episodes
                self.buffer.store_episode([mb_obs, mb_ag, mb_g, mb_actions])
                self._update_normalizer([mb_obs, mb_ag, mb_g, mb_actions])
                for _ in range(self.args.n_batches):
                    # train the network
                    self._update_network()
                # soft update
                self._soft_update_target_network(self.actor_target_network, self.actor_network)
                self._soft_update_target_network(self.critic_target_network, self.critic_network)
            # start to do the evaluation
            success_rate = self._eval_agent()
            if MPI.COMM_WORLD.Get_rank() == 0:
                print('[{}] epoch is: {}, eval success rate is: {:.3f}'.format(datetime.now(), epoch, success_rate))
            models.append(self.actor_network.state_dict())
            o_norms.append(self.o_norm)
            g_norms.append(self.g_norm)
            accuracies.append(success_rate)
        return models,o_norms,g_norms,accuracies

    def get_active_tiles(self, state, action):
        active_tiles = self.tiles(self.iht, self.num_tiles, np.concatenate((state,action)))
        return active_tiles

    def hash_coords(self, coordinates, m, read_only=False):
        if isinstance(m, IHT): return m.get_index(tuple(coordinates), read_only)
        if isinstance(m, int): return hash(tuple(coordinates)) % m
        if m is None: return coordinates

    def tiles(self, iht_or_size, num_tilings, floats, ints=None, read_only=False):
        """returns num-tilings tile indices corresponding to the floats and ints"""
        if ints is None:
            ints = []
        qfloats = [floor(f * num_tilings) for f in floats]
        tiles = []
        for tiling in range(num_tilings):
            tilingX2 = tiling * 2
            coords = [tiling]
            b = tiling
            for q in qfloats:
                coords.append((q + b) // num_tilings)
                b += tilingX2
            coords.extend(ints)
            tiles.append(self.hash_coords(coords, iht_or_size, read_only))
        return tiles

    # no exploration, returns the action
    def no_explore(self, action, state, step):
        return action
        
    # run basic e-greedy exploration
    def e_greedy(self, action, state, step):
        if np.random.random() < self.epsilon:
            # Select a random action state
            action = np.random.uniform(low=-self.env_params['action_max'], high=self.env_params['action_max'], \
                                            size=self.env_params['action'])
        return action
        
    # Use decaying e-greedy strategy
    def e_greedy_decay(self, action, state, step):
        if np.random.random() < self.e_schedule.value(step):
            # Select a random action state
            action = np.random.uniform(low=-self.env_params['action_max'], high=self.env_params['action_max'], \
                                            size=self.env_params['action'])
        return action
    
    # Use upper confidence bound
    def ucb(self, action, state, step):
        active = self.get_active_tiles(np.random.normal(state.numpy().flatten()),np.random.normal(action))
        self.weights_ucb[active] += 1
        additive = np.sqrt(np.log(np.sum(self.weights_ucb[active]))/np.sum(self.weights_ucb))
        return action + np.random.normal(loc=0.0,scale=additive,size=4)
    
    # Select action as a function of how frequently they have occured
    def frequency_explore(self, action, state, step):
        beta = 0.5
        active = self.get_active_tiles(np.random.normal(state.numpy().flatten()),np.random.normal(action))
        self.weights_fq[active] += 1
        additive = beta/np.sqrt(np.sum(self.weights_fq[active]))
        return action + np.random.normal(loc=0.0,scale=additive,size=4)

    # White noise based exploration
    def standard_explore(self, action, state, step):
        # random actions...
        random_actions = np.random.uniform(low=-self.env_params['action_max'], high=self.env_params['action_max'], \
                                            size=self.env_params['action'])
        # choose if use the random actions
        return action + np.random.binomial(1, self.args.random_eps, 1)[0] * (random_actions - action)
    
    # Explore using OU process
    def ornstein_uhlenbeck(self, action, state, step):
        return action + self.ou_schedule.value(step)
    
    # pre_process the inputs
    def _preproc_inputs(self, obs, g):
        obs_norm = self.o_norm.normalize(obs)
        g_norm = self.g_norm.normalize(g)
        # concatenate the stuffs
        inputs = np.concatenate([obs_norm, g_norm])
        inputs = torch.tensor(inputs, dtype=torch.float32).unsqueeze(0)
        if self.args.cuda:
            inputs = inputs.cuda()
        return inputs
    
    # this function will choose action for the agent and do the exploration
    def _select_actions(self, pi, state, step):
        action = pi.cpu().numpy().squeeze()
        # add the gaussian
        #action += self.args.noise_eps * self.env_params['action_max'] * np.random.randn(*action.shape)
        #action = np.clip(action, -self.env_params['action_max'], self.env_params['action_max'])
        action = self.explore_strategy(action,state,step)
        action = np.clip(action, -self.env_params['action_max'], self.env_params['action_max'])
        return action

    # update the normalizer
    def _update_normalizer(self, episode_batch):
        mb_obs, mb_ag, mb_g, mb_actions = episode_batch
        mb_obs_next = mb_obs[:, 1:, :]
        mb_ag_next = mb_ag[:, 1:, :]
        # get the number of normalization transitions
        num_transitions = mb_actions.shape[1]
        # create the new buffer to store them
        buffer_temp = {'obs': mb_obs, 
                       'ag': mb_ag,
                       'g': mb_g, 
                       'actions': mb_actions, 
                       'obs_next': mb_obs_next,
                       'ag_next': mb_ag_next,
                       }
        transitions = self.her_module.sample_her_transitions(buffer_temp, num_transitions)
        obs, g = transitions['obs'], transitions['g']
        # pre process the obs and g
        transitions['obs'], transitions['g'] = preproc_og(obs, g)
        # update
        self.o_norm.update(transitions['obs'])
        self.g_norm.update(transitions['g'])
        # recompute the stats
        self.o_norm.recompute_stats()
        self.g_norm.recompute_stats()

    # soft update
    def _soft_update_target_network(self, target, source):
        for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_((1 - self.args.polyak) * param.data + self.args.polyak * target_param.data)

    # update the network
    def _update_network(self):
        # sample the episodes
        transitions = self.buffer.sample(self.args.batch_size)
        # pre-process the observation and goal
        o, o_next, g = transitions['obs'], transitions['obs_next'], transitions['g']
        transitions['obs'], transitions['g'] = preproc_og(o, g)
        transitions['obs_next'], transitions['g_next'] = preproc_og(o_next, g)
        # start to do the update
        obs_norm = self.o_norm.normalize(transitions['obs'])
        g_norm = self.g_norm.normalize(transitions['g'])
        inputs_norm = np.concatenate([obs_norm, g_norm], axis=1)
        obs_next_norm = self.o_norm.normalize(transitions['obs_next'])
        g_next_norm = self.g_norm.normalize(transitions['g_next'])
        inputs_next_norm = np.concatenate([obs_next_norm, g_next_norm], axis=1)
        # transfer them into the tensor
        inputs_norm_tensor = torch.tensor(inputs_norm, dtype=torch.float32)
        inputs_next_norm_tensor = torch.tensor(inputs_next_norm, dtype=torch.float32)
        actions_tensor = torch.tensor(transitions['actions'], dtype=torch.float32)
        r_tensor = torch.tensor(transitions['r'], dtype=torch.float32) 
        if self.args.cuda:
            inputs_norm_tensor = inputs_norm_tensor.cuda()
            inputs_next_norm_tensor = inputs_next_norm_tensor.cuda()
            actions_tensor = actions_tensor.cuda()
            r_tensor = r_tensor.cuda()
        # calculate the target Q value function
        with torch.no_grad():
            # do the normalization
            # concatenate the stuffs
            actions_next = self.actor_target_network(inputs_next_norm_tensor)
            q_next_value = self.critic_target_network(inputs_next_norm_tensor, actions_next)
            q_next_value = q_next_value.detach()
            target_q_value = r_tensor + self.args.gamma * q_next_value
            target_q_value = target_q_value.detach()
            # clip the q value
            clip_return = 1 / (1 - self.args.gamma)
            target_q_value = torch.clamp(target_q_value, -clip_return, 0)
        # the q loss
        real_q_value = self.critic_network(inputs_norm_tensor, actions_tensor)
        critic_loss = (target_q_value - real_q_value).pow(2).mean()
        # the actor loss
        actions_real = self.actor_network(inputs_norm_tensor)
        actor_loss = -self.critic_network(inputs_norm_tensor, actions_real).mean()
        actor_loss += self.args.action_l2 * (actions_real / self.env_params['action_max']).pow(2).mean()
        # start to update the network
        self.actor_optim.zero_grad()
        actor_loss.backward()
        sync_grads(self.actor_network)
        self.actor_optim.step()
        # update the critic_network
        self.critic_optim.zero_grad()
        critic_loss.backward()
        sync_grads(self.critic_network)
        self.critic_optim.step()

    # do the evaluation
    def _eval_agent(self):
        total_success_rate = []
        for _ in range(self.args.n_test_rollouts):
            per_success_rate = []
            observation = self.env.reset()
            obs = observation['observation']
            g = observation['desired_goal']
            for _ in range(self.env_params['max_timesteps']):
                with torch.no_grad():
                    input_tensor = self._preproc_inputs(obs, g)
                    pi = self.actor_network(input_tensor)
                    # convert the actions
                    actions = pi.detach().cpu().numpy().squeeze()
                observation_new, _, _, info = self.env.step(actions)
                obs = observation_new['observation']
                g = observation_new['desired_goal']
                per_success_rate.append(info['is_success'])
            total_success_rate.append(per_success_rate)
        total_success_rate = np.array(total_success_rate)
        local_success_rate = np.mean(total_success_rate[:, -1])
        global_success_rate = MPI.COMM_WORLD.allreduce(local_success_rate, op=MPI.SUM)
        return global_success_rate / MPI.COMM_WORLD.Get_size()


### Define environmental parameters

In [17]:
def get_env_params(env):
    obs = env.reset()
    # close the environment
    params = {'obs': obs['observation'].shape[0],
            'goal': obs['desired_goal'].shape[0],
            'action': env.action_space.shape[0],
            'action_max': env.action_space.high[0],
            }
    params['max_timesteps'] = env._max_episode_steps
    return params


### Parse arguments and run code

In [19]:
def launch(args, path=None):
    # Save path
    save_path = os.path.join(path,'fetch_models',f'{args.env_name}_{args.strategy}.wts')
    print(save_path)
    # create the ddpg_agent
    env = gym.make(args.env_name)
    # get the environment parameters
    env_params = get_env_params(env)
    # create the ddpg agent to interact with the environment 
    print(env_params)
    '''
    models = []
    accuracies = []
    o_norms = []
    g_norms = []
    t_runs = tqdm.notebook.trange(args.n_runs)
    for run in t_runs:
        t_runs.set_description(f'Executing run: {run}/{args.n_runs}')
        ddpg_trainer = ddpg_agent(args, env, env_params)
        t_models,t_o_norms,t_g_norms,t_accuracies = ddpg_trainer.learn()
        models.append(t_models)
        accuracies.append(deepcopy(t_accuracies))
        o_norms.append(t_o_norms)
        g_norms.append(t_g_norms)
        # Remove model to prevent bad things from happening
        del ddpg_trainer
    # Save weights here
    # torch.save(agent_weights.state_dict(), save_path)
    # Save extra essential pieces
    with open(save_path,'wb') as f:
        pickle.dump([accuracies, models[-1][-1]], f)
    #torch.save([o_norms,g_norms,accuracies], save_path)
    return models,o_norms,g_norms,accuracies
    '''

### Execute various configurations

##### Run 1: No exploration noise

In [20]:
args_to_parse = '--env-name FetchSlide-v1 --n-epochs=200 --n-runs=5 --strategy=no_explore --noisy=0 --cuda'
args = get_args(args_to_parse)
out_no_explore = launch(args, fullPath)

/home/nathaniel/Classes/CS5180_Reinforcement_Learning/ReinforcementRobotExploration/fetch_models/FetchSlide-v1_no_explore.wts
{'obs': 25, 'goal': 3, 'action': 4, 'action_max': 1.0, 'max_timesteps': 50}


##### Run 4: Parameter Space Noise

In [ ]:
args_to_parse = '--env-name FetchSlide-v1 --n-epochs=200 --n-runs=5 --strategy=standard --noisy=1'
args = get_args(args_to_parse)
out_no_explore = launch(args, fullPath)

/home/nathaniel/Classes/CS5180_Reinforcement_Learning/ReinforcementRobotExploration/fetch_models/FetchSlide-v1_standard.wts


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

[2021-12-07 09:00:52.437481] epoch is: 0, eval success rate is: 0.000
[2021-12-07 09:01:18.150044] epoch is: 1, eval success rate is: 0.000
[2021-12-07 09:01:42.935233] epoch is: 2, eval success rate is: 0.000
[2021-12-07 09:02:09.004258] epoch is: 3, eval success rate is: 0.000
[2021-12-07 09:02:34.652098] epoch is: 4, eval success rate is: 0.000
[2021-12-07 09:02:59.933943] epoch is: 5, eval success rate is: 0.000
[2021-12-07 09:03:25.505121] epoch is: 6, eval success rate is: 0.000
[2021-12-07 09:03:50.813030] epoch is: 7, eval success rate is: 0.000
[2021-12-07 09:04:16.918631] epoch is: 8, eval success rate is: 0.000
[2021-12-07 09:04:42.926667] epoch is: 9, eval success rate is: 0.000
[2021-12-07 09:05:08.087194] epoch is: 10, eval success rate is: 0.000
[2021-12-07 09:05:33.467963] epoch is: 11, eval success rate is: 0.000
[2021-12-07 09:05:58.734458] epoch is: 12, eval success rate is: 0.000
[2021-12-07 09:06:26.516938] epoch is: 13, eval success rate is: 0.000
[2021-12-07 09:0

  0%|          | 0/200 [00:00<?, ?it/s]

...

[2021-12-07 12:17:17.674833] epoch is: 0, eval success rate is: 0.000
[2021-12-07 12:17:37.775445] epoch is: 1, eval success rate is: 0.000
[2021-12-07 12:17:58.127475] epoch is: 2, eval success rate is: 0.000
[2021-12-07 12:18:18.542594] epoch is: 3, eval success rate is: 0.000
[2021-12-07 12:18:38.990122] epoch is: 4, eval success rate is: 0.000
[2021-12-07 12:18:59.627085] epoch is: 5, eval success rate is: 0.000
[2021-12-07 12:19:20.204035] epoch is: 6, eval success rate is: 0.000
[2021-12-07 12:19:42.152881] epoch is: 7, eval success rate is: 0.000
[2021-12-07 12:20:03.516359] epoch is: 8, eval success rate is: 0.000
[2021-12-07 12:20:24.343082] epoch is: 9, eval success rate is: 0.000
[2021-12-07 12:20:44.969515] epoch is: 10, eval success rate is: 0.000
[2021-12-07 12:21:05.875383] epoch is: 11, eval success rate is: 0.000
[2021-12-07 12:21:27.785249] epoch is: 12, eval success rate is: 0.000
[2021-12-07 12:21:50.300116] epoch is: 13, eval success rate is: 0.000
[2021-12-07 12:2

  0%|          | 0/200 [00:00<?, ?it/s]

[2021-12-07 13:30:30.415641] epoch is: 0, eval success rate is: 0.000
[2021-12-07 13:30:52.315418] epoch is: 1, eval success rate is: 0.000
[2021-12-07 13:31:13.594499] epoch is: 2, eval success rate is: 0.000
[2021-12-07 13:31:34.698700] epoch is: 3, eval success rate is: 0.000
[2021-12-07 13:31:55.693937] epoch is: 4, eval success rate is: 0.000
[2021-12-07 13:32:17.013535] epoch is: 5, eval success rate is: 0.000
[2021-12-07 13:32:38.198171] epoch is: 6, eval success rate is: 0.000
[2021-12-07 13:32:59.493564] epoch is: 7, eval success rate is: 0.100
[2021-12-07 13:33:20.727858] epoch is: 8, eval success rate is: 0.000
[2021-12-07 13:33:42.050859] epoch is: 9, eval success rate is: 0.100
[2021-12-07 13:34:03.402678] epoch is: 10, eval success rate is: 0.000
[2021-12-07 13:34:24.488293] epoch is: 11, eval success rate is: 0.000
[2021-12-07 13:34:45.431571] epoch is: 12, eval success rate is: 0.000
[2021-12-07 13:35:05.923594] epoch is: 13, eval success rate is: 0.000
[2021-12-07 13:3

  0%|          | 0/200 [00:00<?, ?it/s]

[2021-12-07 14:41:26.579865] epoch is: 0, eval success rate is: 0.000
[2021-12-07 14:41:44.646444] epoch is: 1, eval success rate is: 0.000
[2021-12-07 14:42:02.664528] epoch is: 2, eval success rate is: 0.000
[2021-12-07 14:42:20.933590] epoch is: 3, eval success rate is: 0.000
[2021-12-07 14:42:39.017192] epoch is: 4, eval success rate is: 0.000
[2021-12-07 14:42:57.283607] epoch is: 5, eval success rate is: 0.000
[2021-12-07 14:43:15.768549] epoch is: 6, eval success rate is: 0.000
[2021-12-07 14:43:34.376759] epoch is: 7, eval success rate is: 0.000
[2021-12-07 14:43:52.770406] epoch is: 8, eval success rate is: 0.100
[2021-12-07 14:44:11.416338] epoch is: 9, eval success rate is: 0.000
[2021-12-07 14:44:30.033298] epoch is: 10, eval success rate is: 0.000
[2021-12-07 14:44:48.435459] epoch is: 11, eval success rate is: 0.000
[2021-12-07 14:45:07.017375] epoch is: 12, eval success rate is: 0.000
[2021-12-07 14:45:25.471790] epoch is: 13, eval success rate is: 0.000
[2021-12-07 14:4

##### Run 6: Frequency Based Visit Exploration

In [11]:
args_to_parse = '--env-name FetchSlide-v1 --n-epochs=200 --n-runs=5 --strategy=count --noisy=0'
args = get_args(args_to_parse)
out_no_explore = launch(args, fullPath)

/home/nathaniel/Classes/CS5180_Reinforcement_Learning/ReinforcementRobotExploration/fetch_models/FetchSlide-v1_count.wts


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

IHT full, starting to allow collisions
[2021-12-07 16:37:03.436427] epoch is: 0, eval success rate is: 0.000
[2021-12-07 16:37:23.795174] epoch is: 1, eval success rate is: 0.000
[2021-12-07 16:37:44.883569] epoch is: 2, eval success rate is: 0.000
[2021-12-07 16:38:05.631225] epoch is: 3, eval success rate is: 0.000
[2021-12-07 16:38:25.979368] epoch is: 4, eval success rate is: 0.000
[2021-12-07 16:38:46.634414] epoch is: 5, eval success rate is: 0.000
[2021-12-07 16:39:07.347276] epoch is: 6, eval success rate is: 0.100
[2021-12-07 16:39:27.987729] epoch is: 7, eval success rate is: 0.000
[2021-12-07 16:39:48.918155] epoch is: 8, eval success rate is: 0.000
[2021-12-07 16:40:09.958478] epoch is: 9, eval success rate is: 0.000
[2021-12-07 16:40:30.707460] epoch is: 10, eval success rate is: 0.000
[2021-12-07 16:40:50.992533] epoch is: 11, eval success rate is: 0.000
[2021-12-07 16:41:12.009383] epoch is: 12, eval success rate is: 0.000
[2021-12-07 16:41:33.056998] epoch is: 13, eval 

  0%|          | 0/200 [00:00<?, ?it/s]

...

IHT full, starting to allow collisions
[2021-12-07 19:16:12.562865] epoch is: 0, eval success rate is: 0.000
[2021-12-07 19:16:32.952893] epoch is: 1, eval success rate is: 0.000
[2021-12-07 19:16:54.712073] epoch is: 2, eval success rate is: 0.000
[2021-12-07 19:17:15.850196] epoch is: 3, eval success rate is: 0.000
[2021-12-07 19:17:37.903476] epoch is: 4, eval success rate is: 0.000
[2021-12-07 19:17:59.403430] epoch is: 5, eval success rate is: 0.000
[2021-12-07 19:18:20.140563] epoch is: 6, eval success rate is: 0.000
[2021-12-07 19:18:41.552090] epoch is: 7, eval success rate is: 0.000
[2021-12-07 19:19:02.735357] epoch is: 8, eval success rate is: 0.000
[2021-12-07 19:19:24.448678] epoch is: 9, eval success rate is: 0.000
[2021-12-07 19:19:45.711826] epoch is: 10, eval success rate is: 0.000
[2021-12-07 19:20:06.129094] epoch is: 11, eval success rate is: 0.000
[2021-12-07 19:20:27.439620] epoch is: 12, eval success rate is: 0.000
[2021-12-07 19:20:49.078226] epoch is: 13, eval 

  0%|          | 0/200 [00:00<?, ?it/s]

IHT full, starting to allow collisions
[2021-12-07 20:35:51.365848] epoch is: 0, eval success rate is: 0.000
[2021-12-07 20:36:13.343811] epoch is: 1, eval success rate is: 0.000
[2021-12-07 20:36:34.616662] epoch is: 2, eval success rate is: 0.000
[2021-12-07 20:36:56.371262] epoch is: 3, eval success rate is: 0.000
[2021-12-07 20:37:17.858245] epoch is: 4, eval success rate is: 0.000
[2021-12-07 20:37:38.503185] epoch is: 5, eval success rate is: 0.100
[2021-12-07 20:37:59.515501] epoch is: 6, eval success rate is: 0.000
[2021-12-07 20:38:20.879317] epoch is: 7, eval success rate is: 0.000
[2021-12-07 20:38:41.518369] epoch is: 8, eval success rate is: 0.000
[2021-12-07 20:39:02.345708] epoch is: 9, eval success rate is: 0.000
[2021-12-07 20:39:23.784423] epoch is: 10, eval success rate is: 0.000
[2021-12-07 20:39:45.161840] epoch is: 11, eval success rate is: 0.000
[2021-12-07 20:40:06.534385] epoch is: 12, eval success rate is: 0.100
[2021-12-07 20:40:27.727176] epoch is: 13, eval 

  0%|          | 0/200 [00:00<?, ?it/s]

IHT full, starting to allow collisions
[2021-12-07 21:50:02.275572] epoch is: 0, eval success rate is: 0.000
[2021-12-07 21:50:19.094435] epoch is: 1, eval success rate is: 0.000
[2021-12-07 21:50:36.250333] epoch is: 2, eval success rate is: 0.000
[2021-12-07 21:50:53.409908] epoch is: 3, eval success rate is: 0.000
[2021-12-07 21:51:10.459929] epoch is: 4, eval success rate is: 0.000
[2021-12-07 21:51:27.418018] epoch is: 5, eval success rate is: 0.000
[2021-12-07 21:51:44.392173] epoch is: 6, eval success rate is: 0.000
[2021-12-07 21:52:01.338398] epoch is: 7, eval success rate is: 0.000
[2021-12-07 21:52:18.193467] epoch is: 8, eval success rate is: 0.000
[2021-12-07 21:52:35.301594] epoch is: 9, eval success rate is: 0.000
[2021-12-07 21:52:52.581688] epoch is: 10, eval success rate is: 0.000
[2021-12-07 21:53:09.883044] epoch is: 11, eval success rate is: 0.000
[2021-12-07 21:53:27.278635] epoch is: 12, eval success rate is: 0.000
[2021-12-07 21:53:44.414570] epoch is: 13, eval 

##### Run 8: Standard Exploration

In [12]:
args_to_parse = '--env-name FetchSlide-v1 --n-epochs=200 --n-runs=5 --strategy=standard --noisy=0 --cuda'
args = get_args(args_to_parse)
out_no_explore = launch(args, fullPath)

/home/nathaniel/Classes/CS5180_Reinforcement_Learning/ReinforcementRobotExploration/fetch_models/FetchSlide-v1_standard.wts


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

[2021-12-07 23:24:31.164566] epoch is: 0, eval success rate is: 0.000
[2021-12-07 23:24:49.026595] epoch is: 1, eval success rate is: 0.000
[2021-12-07 23:25:07.339756] epoch is: 2, eval success rate is: 0.000
[2021-12-07 23:25:24.934357] epoch is: 3, eval success rate is: 0.000
[2021-12-07 23:25:42.691876] epoch is: 4, eval success rate is: 0.000
[2021-12-07 23:26:00.459074] epoch is: 5, eval success rate is: 0.000
[2021-12-07 23:26:18.776828] epoch is: 6, eval success rate is: 0.000
[2021-12-07 23:26:36.305175] epoch is: 7, eval success rate is: 0.000
[2021-12-07 23:26:54.440769] epoch is: 8, eval success rate is: 0.000
[2021-12-07 23:27:13.279227] epoch is: 9, eval success rate is: 0.000
[2021-12-07 23:27:31.730853] epoch is: 10, eval success rate is: 0.000
[2021-12-07 23:27:51.260241] epoch is: 11, eval success rate is: 0.000
[2021-12-07 23:28:09.497890] epoch is: 12, eval success rate is: 0.000
[2021-12-07 23:28:28.094008] epoch is: 13, eval success rate is: 0.000
[2021-12-07 23:2

  0%|          | 0/200 [00:00<?, ?it/s]

...

[2021-12-08 01:25:33.003151] epoch is: 0, eval success rate is: 0.000
[2021-12-08 01:25:50.805143] epoch is: 1, eval success rate is: 0.000
[2021-12-08 01:26:09.215849] epoch is: 2, eval success rate is: 0.000
[2021-12-08 01:26:27.265475] epoch is: 3, eval success rate is: 0.000
[2021-12-08 01:26:45.261542] epoch is: 4, eval success rate is: 0.000
[2021-12-08 01:27:03.162387] epoch is: 5, eval success rate is: 0.000
[2021-12-08 01:27:21.031833] epoch is: 6, eval success rate is: 0.000
[2021-12-08 01:27:39.601218] epoch is: 7, eval success rate is: 0.000
[2021-12-08 01:27:57.472712] epoch is: 8, eval success rate is: 0.000
[2021-12-08 01:28:16.334435] epoch is: 9, eval success rate is: 0.000
[2021-12-08 01:28:34.400106] epoch is: 10, eval success rate is: 0.000
[2021-12-08 01:28:52.928691] epoch is: 11, eval success rate is: 0.000
[2021-12-08 01:29:11.501199] epoch is: 12, eval success rate is: 0.000
[2021-12-08 01:29:30.145005] epoch is: 13, eval success rate is: 0.000
[2021-12-08 01:2

  0%|          | 0/200 [00:00<?, ?it/s]

[2021-12-08 02:26:16.653122] epoch is: 0, eval success rate is: 0.000
[2021-12-08 02:26:34.598661] epoch is: 1, eval success rate is: 0.000
[2021-12-08 02:26:52.812629] epoch is: 2, eval success rate is: 0.000
[2021-12-08 02:27:11.301911] epoch is: 3, eval success rate is: 0.000
[2021-12-08 02:27:29.926901] epoch is: 4, eval success rate is: 0.100
[2021-12-08 02:27:48.127309] epoch is: 5, eval success rate is: 0.000
[2021-12-08 02:28:06.056343] epoch is: 6, eval success rate is: 0.000
[2021-12-08 02:28:24.642654] epoch is: 7, eval success rate is: 0.000
[2021-12-08 02:28:42.645973] epoch is: 8, eval success rate is: 0.000
[2021-12-08 02:29:00.705372] epoch is: 9, eval success rate is: 0.000
[2021-12-08 02:29:18.407878] epoch is: 10, eval success rate is: 0.000
[2021-12-08 02:29:36.065658] epoch is: 11, eval success rate is: 0.000
[2021-12-08 02:29:54.218380] epoch is: 12, eval success rate is: 0.000
[2021-12-08 02:30:12.466160] epoch is: 13, eval success rate is: 0.000
[2021-12-08 02:3

  0%|          | 0/200 [00:00<?, ?it/s]

[2021-12-08 03:26:57.872225] epoch is: 0, eval success rate is: 0.000
[2021-12-08 03:27:15.406216] epoch is: 1, eval success rate is: 0.000
[2021-12-08 03:27:33.230183] epoch is: 2, eval success rate is: 0.000
[2021-12-08 03:27:51.745433] epoch is: 3, eval success rate is: 0.000
[2021-12-08 03:28:10.111616] epoch is: 4, eval success rate is: 0.000
[2021-12-08 03:28:27.782823] epoch is: 5, eval success rate is: 0.000
[2021-12-08 03:28:46.110470] epoch is: 6, eval success rate is: 0.000
[2021-12-08 03:29:04.540592] epoch is: 7, eval success rate is: 0.000
[2021-12-08 03:29:22.278852] epoch is: 8, eval success rate is: 0.000
[2021-12-08 03:29:39.919863] epoch is: 9, eval success rate is: 0.000
[2021-12-08 03:29:57.691636] epoch is: 10, eval success rate is: 0.000
[2021-12-08 03:30:15.707853] epoch is: 11, eval success rate is: 0.000
[2021-12-08 03:30:34.034629] epoch is: 12, eval success rate is: 0.100
[2021-12-08 03:30:51.752643] epoch is: 13, eval success rate is: 0.000
[2021-12-08 03:3

In [ ]:
#  Jupyter UI - Trigger policies for individual agents
def button_callback(button):
    for b in buttons:
        b.disabled = True

    env = envs[button.description]['model']
    final_policy = None
    try:
        env_name = envs[button.description]['model'].env.spec.id
        env_params = get_env_params(env)
        model = torch.load(f'{env_name}.wts')
        model_weights = model[4]
        actor_model = actor(env_params)
        actor_model.load_state_dict(model_weights)
        # Create policy using function
        def policy(state):
            # reset the environment
            obs = state['observation']
            g = state['desired_goal']
            o_norm = normalizer(size=10, default_clip_range=200)
            o_norm.mean = model[0]
            o_norm.std = model[1]
            g_norm = normalizer(size=3, default_clip_range=200)
            g_norm.mean = model[2]
            g_norm.std = model[3]
            obs_norm = o_norm.normalize(obs)
            g_norm = g_norm.normalize(g)
            # concatenate the stuffs
            input_tensor = np.concatenate([obs_norm, g_norm])
            input_tensor = torch.tensor(input_tensor, dtype=torch.float32).unsqueeze(0)
            with torch.no_grad():
                pi = actor_model(input_tensor)
            action = pi.cpu().numpy().squeeze()
            # add the gaussian
            action = np.clip(action, -1, 1)
            return action
        final_policy = policy
    except Exception as e:
        print('Could not create policy - running random policy')
        print(str(e))
        traceback.print_exc()
        return
    render(env, final_policy)
    env.close()
        
    for b in buttons:
        b.disabled = False

buttons = []
for env_id in envs.keys():
    button = widgets.Button(description=env_id)
    button.on_click(button_callback)
    buttons.append(button)

print('Click a button to evaluate a policy')
b = widgets.HBox(buttons)
display(b)